# 🧠 Web Content Summarizer

This notebook extracts and summarizes text content from websites using OpenAI-compatible APIs (Groq or Ollama).

## Features:
- Scrapes content using `requests` or `Selenium`
- Uses OpenAI-compatible models like `grok` & Local models like `Ollama` to generate summaries
- Supports custom URLs


In [1]:
import os
import requests
import json
import socket
import ipaddress
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from IPython.display import display, Markdown, update_display
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

## 🔧 1. Load API Keys
We load the API key from a `.env` file to securely connect to the Groq or Ollama LLM backend.


In [2]:
load_dotenv(override=True)
api_key  = os.getenv("GROQ_API_KEY")

if not api_key:
    print("No API_Key found, Please set the API_KEY.")
    exit(1)
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")



API key found and looks good so far!


In [3]:
openai = OpenAI(api_key=api_key, base_url = "https://api.groq.com/openai/v1")
ollama_with_openai = OpenAI(api_key = "ollama", base_url = "http://localhost:11434/v1")

## 🌐 2. Website Content Extractor
This class retrieves the raw HTML and title of a webpage using `requests` or Selenium.


In [4]:
def is_safe_url(url):
    try:
        parsed = urlparse(url)
        if parsed.scheme not in ["http", "https"] or parsed.netloc == "":
            return False

        host = parsed.hostname
        ip = ipaddress.ip_address(socket.gethostbyname(host))
        if ip.is_private or ip.is_loopback or ip.is_reserved or ip.is_link_local:
            return False
    except Exception:
        return False
    return True


class Web_Scapper:
    """
    A utility class to represent a Website that we have scraped, using Selenium, with extracted links.
    """

    def __init__(self, url):
        if not is_safe_url(url):
            raise ValueError("Invalid or unsafe URL")

        self.url = url
        self.title = "No title found"
        self.text = ""
        self.links = []

        # Setup Selenium
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-extensions")
        options.add_argument("--disable-blink-features=AutomationControlled")

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        try:
            driver.set_page_load_timeout(20)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, "html.parser")

            # Get title
            self.title = soup.title.string.strip() if soup.title else "No title found"

            # Remove irrelevant tags
            if soup.body:
                for tag in soup.body(["script", "style", "img", "input"]):
                    tag.decompose()
                self.text = soup.body.get_text(separator="\n", strip=True)

            # Extract all valid links
            all_links = [a.get("href") for a in soup.find_all("a") if a.get("href")]
            self.links = all_links

        except WebDriverException as e:
            print(f"Error loading page with Selenium: {e}")
        finally:
            driver.quit()

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\n\nWebpage Contents:\n{self.text}\n\n"


In [5]:
website = Web_Scapper("https://edwarddonner.com")
# print(web_scrapper.get_contents())
print(website.get_contents())
# web_scrapper.links


Webpage Title:
Home - Edward Donner

Webpage Contents:
Skip to content
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has 

## 📝 3. Prompt Engineering and Summarization
This section defines the prompts used to instruct the `large language model (LLM)` for summarization and provides the core functions to perform the website summarization using the `selected LLM (Groq or Ollama)`.


In [6]:
system_prompt = "You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown."

def user_prompt_for(website):
    user_prompt = f"- You are looking at a website titled: '{website.title}'.\n\n- The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n{website.text}"
    return user_prompt
# print(user_prompt_for(content))


In [7]:
# And now: call the OpenAI API. You will get very familiar with this!
def summarize(url):
    website = Web_Scapper(url)
    response = openai.chat.completions.create(
        model = "llama3-70b-8192",
        messages = [
        {"role": "system", "content": system_prompt}
        ,{"role": "user", "content": user_prompt_for(website)}
    ]
    )
    return response.choices[0].message.content


# A function to display this nicely in the Jupyter output, using markdown
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))


In [10]:
display_summary("https://www.gopalinfo.com/")

**Gopal Info Website Summary**
================================

Gopal Info is a digital growth agency that offers a range of services to help businesses thrive online. Their expert solutions include:

* **Digital Design**: Graphic Designing, UI/UX Design, Branding Consultation, and Web Designing
* **Development**: Web Development and Mobile App Development
* **Marketing**: Social Media Marketing, Search Engine Optimization, and Google Ads

The agency utilizes advanced technologies to deliver high-quality services and follows an agile methodology to ensure project success. They prioritize collaboration, creativity, and quality at every step.

**No notable news or announcements on the website.**

In [9]:
display_summary("https://techcrunch.com")

**TechCrunch Website Summary**
===========================

TechCrunch is a website that provides news, articles, and insights on the tech industry, focusing on startups, venture capital, and technology. The website features various sections, including:

### Latest News

* Cluely's Roy Lee discusses cheating detectors
* Microsoft shares $500M in AI savings internally
* California lawmaker pushes for mandated AI safety reports
* Neobank Revolut seeks $65B valuation
* YouTube prepares crackdown on 'mass-produced' and 'repetitive' videos

### Topics

* AI
* Startups
* Venture
* Apple
* Security
* Events
* Podcasts
* Newsletters

### Upcoming Events

* TechCrunch All Stage 2025 (July 15, Boston)
* TechCrunch Disrupt 2025 (October 27-29, San Francisco)
* StrictlyVC Palo Alto (December 3, Palo Alto)

### Newsletters

* TechCrunch Daily News
* Startups Weekly
* TechCrunch Week in Review
* TechCrunch Mobility

### Video

* Interviews and talks on various topics, including crypto, hardware, startups, and AI

Overall, TechCrunch provides a comprehensive platform for staying updated on the latest developments in the tech industry, with a focus on innovation, entrepreneurship, and investment.